In [3]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from datetime import datetime as dt
import pandas as pd
from db.db_operations import execute_db_operations

from generators.full_generators import (
    create_company_data,
    create_all_erp_data,
    create_mapping_between_all, 
)

company_name = "Lego"

In [4]:
data = create_company_data(company_name=company_name, save_to_csv=True)

Generating data for company: Lego...
Context data for Lego generated: {'company_name': 'Lego', 'count_employee': 400, 'count_department': 20, 'count_customer': 50, 'count_product': 150, 'count_procurement': 120, 'count_service': 100, 'count_account': 60, 'count_vendor': 60, 'estimated_product': 12000000, 'estimated_service': 8000000, 'estimated_overhead': 5000000, 'estimated_revenue': 14000000000}
Generating dimensions for company: Lego.
This usually takes 2-5 mins.

=== Attempt 1 ===
✔ Roles and Names generated.
✔ Procurement data generated.
✔ Services data generated.
✔ Products data generated.
✔ Customers data generated.
✔ Departments data generated.
✔ Vendors data generated.
✔ Business Units and company data generated.
✔ Payroll data generated.
✔ All table lengths validated.
✔ All CSVs saved to: data/outputdata

 * Semantic mapping started * :
Time estimate: 3-5 minutes
✔ Procurement mapping done!
✔ Service mapping done!
✔ All mapping data generated.
✔ All mapping CSVs saved to: dat

In [58]:
gl = pd.read_csv(f"data/outputdata/fact/general_ledger.csv")
#gl_actual = gl[gl["version"] == "Actual"]

#gl_actual.drop(columns=["version", "debit_credit"], inplace=True)

df_service = pd.read_csv(f"data/outputdata/dimensions/service.csv")
df_bu = pd.read_csv(f"data/outputdata/dimensions/bu.csv")
df_vendor = pd.read_csv(f"data/outputdata/dimensions/vendor.csv")
df_customer = pd.read_csv(f"data/outputdata/dimensions/customer.csv")
df_account = pd.read_csv(f"data/outputdata/dimensions/account.csv")
df_procurement = pd.read_csv(f"data/outputdata/dimensions/procurement.csv")
df_products = pd.read_csv(f"data/outputdata/dimensions/product.csv")

C:\Users\jsteensgaard\AppData\Local\Temp\ipykernel_31176\2611534601.py:1: DtypeWarning: Columns (0,1,6,7,8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  gl = pd.read_csv(f"data/outputdata/fact/general_ledger.csv")


In [59]:
gl_actual = gl.copy()

In [60]:
gl_actual = gl_actual.merge(df_bu[["name", "companyname", "companycode"]], left_on="bu_id", right_on="name", how="left")
gl_actual = gl_actual.merge(df_vendor[["name", "vendor_id"]], left_on="vendor_id", right_on="name", how="left")
gl_actual = gl_actual.merge(df_customer[["name", "customer_id"]], left_on="customer_id", right_on="name", how="left")
df_account.rename(columns={"name": "account_name"}, inplace=True)
gl_actual = gl_actual.merge(df_account[["account_name", "AccountKey"]], left_on="account_id", right_on="account_name", how="left")
gl_actual["amount"] = gl_actual["amount"].astype(float)*10
gl_actual.drop(columns=["account_id", "name_x", "name_y", "name",], inplace=True)

In [61]:
df_procurement.rename(columns={"name": "procurement_name"}, inplace=True)
df_procurement["procurement_id"] = df_procurement.index + 1000

df_service.rename(columns={"name": "service_name"}, inplace=True)
df_service["service_id"] = df_service.index + 5000

df_products.rename(columns={"name": "product_name"}, inplace=True)
df_products["product_id"] = df_products.index + 2000

In [62]:
gl_actual = gl_actual.merge(df_procurement[["procurement_name", "procurement_id"]], left_on="procurement_id", right_on="procurement_name", how="left")
gl_actual = gl_actual.merge(df_service[["service_name", "service_id"]], left_on="service_id", right_on="service_name", how="left")
gl_actual = gl_actual.merge(df_products[["product_name", "product_id"]], left_on="product_id", right_on="product_name", how="left")

In [63]:
gl_actual

,document_number,debit_credit,date,amount,quantity,product_id_x,procurement_id_x,service_id_x,bu_id,vendor_id_x,...,vendor_id_y,customer_id_y,account_name,AccountKey,procurement_name,procurement_id_y,service_name,service_id_y,product_name,product_id_y
0,78001000,NaN,2025-06-22,-458269.3,7.0,NaN,ABS resin pellets,NaN,LEGO_DE_ASSEMBLY,BASF Plastics,...,20.0,NaN,Retail COS,4003,ABS resin pellets,1000.0,NaN,NaN,NaN,NaN
1,78001160,NaN,2024-11-20,-458611.3,8.0,NaN,ABS resin pellets,NaN,LEGO_DE_ASSEMBLY,BASF Plastics,...,20.0,NaN,Retail COS,4003,ABS resin pellets,1000.0,NaN,NaN,NaN,NaN
2,78001078,NaN,2024-12-11,-229576.0,8.0,NaN,ABS resin pellets,NaN,LEGO_DE_ASSEMBLY,BASF Plastics,...,20.0,NaN,Retail COS,4003,ABS resin pellets,1000.0,NaN,NaN,NaN,NaN
3,78001090,NaN,2025-07-31,-229294.1,4.0,NaN,ABS resin pellets,NaN,LEGO_DE_ASSEMBLY,BASF Plastics,...,20.0,NaN,Retail COS,4003,ABS resin pellets,1000.0,NaN,NaN,NaN,NaN
4,78001066,NaN,2024-07-11,-229284.5,6.0,NaN,ABS resin pellets,NaN,LEGO_DE_ASSEMBLY,BASF Plastics,...,20.0,NaN,Retail COS,4003,ABS resin pellets,1000.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150220,BUDG78002399,Debit,2025-12-01,1000.0,3.0,LEGO Reusable Bags,NaN,NaN,LEGO_US_PACKAGING,NaN,...,NaN,11.0,Retail Gross Sales,4001,NaN,NaN,NaN,NaN,LEGO Reusable Bags,2094.0
1150221,BUDG78002399,Debit,2025-12-01,5000.0,2.0,LEGO Spider-Man Sets,NaN,NaN,LEGO_SE_THEME,NaN,...,NaN,11.0,Retail Gross Sales,4001,NaN,NaN,NaN,NaN,LEGO Spider-Man Sets,2016.0
1150222,BUDG78002399,Debit,2025-12-01,4000.0,10.0,LEGO Stickers,NaN,NaN,LEGO_US_PACKAGING,NaN,...,NaN,11.0,Retail Gross Sales,4001,NaN,NaN,NaN,NaN,LEGO Stickers,2047.0
1150223,BUDG78002399,Debit,2025-12-01,17000.0,2.0,LEGO Super Mario Sets,NaN,NaN,LEGO_SE_THEME,NaN,...,NaN,11.0,Retail Gross Sales,4001,NaN,NaN,NaN,NaN,LEGO Super Mario Sets,2009.0


In [64]:
#where debit_credit == nan, we fill either Debit or Credit depending on the sign of amount
gl_actual['debit_credit'] = gl_actual.apply(lambda row: 'Debit' if pd.isna(row['debit_credit']) and row['amount'] >= 0 else ('Credit' if pd.isna(row['debit_credit']) and row['amount'] < 0 else row['debit_credit']), axis=1) 

In [65]:
gl_actual_new = gl_actual.copy()

In [67]:
gl_actual_new.drop(columns=["product_id_x", "procurement_id_x", "service_id_x", "vendor_id_x", "customer_id_x", "companyname", "account_name"], inplace=True)

In [69]:
gl_actual_new.drop(columns=["product_name", "service_name", "procurement_name"], inplace=True)

In [70]:
gl_actual_new.rename(columns={"vendor_id_y": "vendor_id", "customer_id_y": "customer_id", "AccountKey": "account_id",
                              "service_id_y": "service_id", "product_id_y": "product_id", "procurement_id_y": "procurement_id"}, inplace=True)

In [72]:
gl_actual_new.drop(columns=["companycode"], inplace=True)

In [73]:
df_bu.rename(columns={"name": "business_unit"}, inplace=True)

In [74]:
gl_actual_new.rename(columns={"bu_id": "business_unit"}, inplace=True)

In [75]:
gl_actual_new

,document_number,debit_credit,date,amount,quantity,business_unit,version,vendor_id,customer_id,account_id,procurement_id,service_id,product_id
0,78001000,Credit,2025-06-22,-458269.3,7.0,LEGO_DE_ASSEMBLY,Actual,20.0,NaN,4003,1000.0,NaN,NaN
1,78001160,Credit,2024-11-20,-458611.3,8.0,LEGO_DE_ASSEMBLY,Actual,20.0,NaN,4003,1000.0,NaN,NaN
2,78001078,Credit,2024-12-11,-229576.0,8.0,LEGO_DE_ASSEMBLY,Actual,20.0,NaN,4003,1000.0,NaN,NaN
3,78001090,Credit,2025-07-31,-229294.1,4.0,LEGO_DE_ASSEMBLY,Actual,20.0,NaN,4003,1000.0,NaN,NaN
4,78001066,Credit,2024-07-11,-229284.5,6.0,LEGO_DE_ASSEMBLY,Actual,20.0,NaN,4003,1000.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150220,BUDG78002399,Debit,2025-12-01,1000.0,3.0,LEGO_US_PACKAGING,Budget,NaN,11.0,4001,NaN,NaN,2094.0
1150221,BUDG78002399,Debit,2025-12-01,5000.0,2.0,LEGO_SE_THEME,Budget,NaN,11.0,4001,NaN,NaN,2016.0
1150222,BUDG78002399,Debit,2025-12-01,4000.0,10.0,LEGO_US_PACKAGING,Budget,NaN,11.0,4001,NaN,NaN,2047.0
1150223,BUDG78002399,Debit,2025-12-01,17000.0,2.0,LEGO_SE_THEME,Budget,NaN,11.0,4001,NaN,NaN,2009.0


In [76]:
gl_actual_new.to_csv(f"data/cbs_data/general_ledger.csv", index=False)

In [78]:
df_bu.to_csv(f"data/cbs_data/dim_business_unit.csv", index=False)

In [80]:
df_service.drop(columns=["proportionality"], inplace=True)
df_procurement.drop(columns=["proportionality"], inplace=True)
df_products.drop(columns=["proportionality"], inplace=True)

In [81]:
df_service.to_csv(f"data/cbs_data/dim_service.csv", index=False)
df_procurement.to_csv(f"data/cbs_data/dim_procurement.csv", index=False)
df_products.to_csv(f"data/cbs_data/dim_product.csv", index=False)

In [82]:
df_vendor.drop(columns=["proportionality"], inplace=True)
df_vendor.rename(columns={"name": "vendor_name"}, inplace=True)
df_vendor.to_csv(f"data/cbs_data/dim_vendor.csv", index=False)

In [83]:
df_customer.drop(columns=["proportionality"], inplace=True)
df_customer.rename(columns={"name": "customer_name"}, inplace=True)
df_customer.to_csv(f"data/cbs_data/dim_customer.csv", index=False)

In [88]:
#df_account.rename(columns={"AccountKey": "account_id"}, inplace=True)
df_account.to_csv(f"data/cbs_data/dim_CoA.csv", index=False)

In [89]:
gl_actual_new = gl_actual_new.merge(df_vendor, on="vendor_id", how="left")

In [91]:
gl_actual_new = gl_actual_new.merge(df_customer, on="customer_id", how="left")
gl_actual_new = gl_actual_new.merge(df_service, on="service_id", how="left")
gl_actual_new = gl_actual_new.merge(df_procurement, on="procurement_id", how="left")
gl_actual_new = gl_actual_new.merge(df_products, on="product_id", how="left")

In [94]:
gl_actual_new.to_csv(f"data/cbs_data/large_general_ledger.csv", index=False)

In [ ]:
gl_actual.rename(columns={"product_id": "product_name", "service_id": "service_name", "procurement_id": "procurement_name",
                          "vendor_id_x": "vendor_name", "customer_id_x": "customer_name",
                          "vendor_id_y": "vendor_id", "customer_id_y": "customer_id", 
                          "companyname": "company_name", "companycode": "company_code",
                          "AccountKey": "account_id", "bu_id": "business_unit", "ammount": "amount_DKK"
                          }, inplace=True)
reordered_columns = ["date", "company_name", "company_code", "business_unit", "account_id", "account_name",
                     "vendor_id", "vendor_name", "customer_id", "customer_name",
                     "product_name", "service_name", "procurement_name"]


In [ ]:
gl_actual =gl_actual[reordered_columns]
gl_actual.drop(columns=["account_name", ], inplace=True)

,date,company_name,company_code,business_unit,account_id,account_name,vendor_id,vendor_name,customer_id,customer_name,product_name,service_name,procurement_name
0,2025-06-22,Lego Germany,1002,LEGO_DE_ASSEMBLY,4003,Retail COS,20.0,BASF Plastics,NaN,NaN,NaN,NaN,ABS resin pellets
1,2024-11-20,Lego Germany,1002,LEGO_DE_ASSEMBLY,4003,Retail COS,20.0,BASF Plastics,NaN,NaN,NaN,NaN,ABS resin pellets
2,2024-12-11,Lego Germany,1002,LEGO_DE_ASSEMBLY,4003,Retail COS,20.0,BASF Plastics,NaN,NaN,NaN,NaN,ABS resin pellets
3,2025-07-31,Lego Germany,1002,LEGO_DE_ASSEMBLY,4003,Retail COS,20.0,BASF Plastics,NaN,NaN,NaN,NaN,ABS resin pellets
4,2024-07-11,Lego Germany,1002,LEGO_DE_ASSEMBLY,4003,Retail COS,20.0,BASF Plastics,NaN,NaN,NaN,NaN,ABS resin pellets
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150220,2025-12-01,Lego USA,1003,LEGO_US_PACKAGING,4001,Retail Gross Sales,NaN,NaN,11.0,Walmart,LEGO Reusable Bags,NaN,NaN
1150221,2025-12-01,Lego Sweden,1001,LEGO_SE_THEME,4001,Retail Gross Sales,NaN,NaN,11.0,Walmart,LEGO Spider-Man Sets,NaN,NaN
1150222,2025-12-01,Lego USA,1003,LEGO_US_PACKAGING,4001,Retail Gross Sales,NaN,NaN,11.0,Walmart,LEGO Stickers,NaN,NaN
1150223,2025-12-01,Lego Sweden,1001,LEGO_SE_THEME,4001,Retail Gross Sales,NaN,NaN,11.0,Walmart,LEGO Super Mario Sets,NaN,NaN


In [102]:
gl_actual_new_actual = gl_actual_new[gl_actual_new["version"] == "Actual"]
gl_actual_new_actual.drop(columns=["version"], inplace=True) 
gl_actual_new_actual.drop(columns=["unit_price_x", "unit_price_y"], inplace=True)
#gl_actual_new_actual.to_csv(f"data/cbs_data/general_ledger_actual.csv", index=False)

C:\Users\jsteensgaard\AppData\Local\Temp\ipykernel_31176\4148513758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gl_actual_new_actual.drop(columns=["version"], inplace=True)
C:\Users\jsteensgaard\AppData\Local\Temp\ipykernel_31176\4148513758.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gl_actual_new_actual.drop(columns=["unit_price_x", "unit_price_y"], inplace=True)


In [107]:
gl_actual_new_actual = gl_actual_new_actual.merge(df_bu[["business_unit", "companyname",  "companycode"]], left_on="business_unit", right_on="business_unit", how="left")

In [108]:
gl_actual_new_actual

,document_number,debit_credit,date,amount,quantity,business_unit,vendor_id,customer_id,account_id,procurement_id,...,vendor_name,vendor_segment,customer_name,customer_segment,service_name,procurement_name,product_name,unit_price,companyname,companycode
0,78001000,Credit,2025-06-22,-458269.3,7.0,LEGO_DE_ASSEMBLY,20.0,NaN,4003,1000.0,...,BASF Plastics,Raw Materials,NaN,NaN,NaN,ABS resin pellets,NaN,NaN,Lego Germany,1002
1,78001160,Credit,2024-11-20,-458611.3,8.0,LEGO_DE_ASSEMBLY,20.0,NaN,4003,1000.0,...,BASF Plastics,Raw Materials,NaN,NaN,NaN,ABS resin pellets,NaN,NaN,Lego Germany,1002
2,78001078,Credit,2024-12-11,-229576.0,8.0,LEGO_DE_ASSEMBLY,20.0,NaN,4003,1000.0,...,BASF Plastics,Raw Materials,NaN,NaN,NaN,ABS resin pellets,NaN,NaN,Lego Germany,1002
3,78001090,Credit,2025-07-31,-229294.1,4.0,LEGO_DE_ASSEMBLY,20.0,NaN,4003,1000.0,...,BASF Plastics,Raw Materials,NaN,NaN,NaN,ABS resin pellets,NaN,NaN,Lego Germany,1002
4,78001066,Credit,2024-07-11,-229284.5,6.0,LEGO_DE_ASSEMBLY,20.0,NaN,4003,1000.0,...,BASF Plastics,Raw Materials,NaN,NaN,NaN,ABS resin pellets,NaN,NaN,Lego Germany,1002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801468,78001099,Credit,2024-04-29,-270972.1,9.0,LEGO_DE_ENGINEERING,NaN,65.0,4007,NaN,...,NaN,NaN,LEGO_DE_ASSEMBLY,Intercompany,Annual Workplace Ergonomics Assessment,NaN,NaN,NaN,Lego Germany,1002
801469,78001021,Debit,2024-04-29,271119.2,4.0,LEGO_DE_ENGINEERING,NaN,65.0,4007,NaN,...,NaN,NaN,LEGO_DE_ASSEMBLY,Intercompany,Annual Workplace Ergonomics Assessment,NaN,NaN,NaN,Lego Germany,1002
801470,78001021,Credit,2024-04-29,-271119.2,4.0,LEGO_DE_ENGINEERING,NaN,65.0,4007,NaN,...,NaN,NaN,LEGO_DE_ASSEMBLY,Intercompany,Annual Workplace Ergonomics Assessment,NaN,NaN,NaN,Lego Germany,1002
801471,78001191,Debit,2024-03-19,542657.3,7.0,LEGO_DE_ENGINEERING,NaN,65.0,4007,NaN,...,NaN,NaN,LEGO_DE_ASSEMBLY,Intercompany,Annual Workplace Ergonomics Assessment,NaN,NaN,NaN,Lego Germany,1002


In [109]:
gl_actual_new_actual.to_csv(f"data/cbs_data/general_ledger_actual.csv", index=False)

In [84]:
gl = pd.read_csv(f"data/outputdata/fact/general_ledger.csv")
gl_actual = gl[gl["version"] == "Actual"]

gl_actual.drop(columns=["version", "debit_credit"], inplace=True)

df_service = pd.read_csv(f"data/outputdata/dimensions/service.csv")
df_bu = pd.read_csv(f"data/outputdata/dimensions/bu.csv")
df_vendor = pd.read_csv(f"data/outputdata/dimensions/vendor.csv")
df_customer = pd.read_csv(f"data/outputdata/dimensions/customer.csv")
df_account = pd.read_csv(f"data/outputdata/dimensions/account.csv")
df_procurement = pd.read_csv(f"data/outputdata/dimensions/procurement.csv")
df_products = pd.read_csv(f"data/outputdata/dimensions/product.csv")

gl_actual = gl_actual.merge(df_bu[["name", "companyname", "companycode"]], left_on="bu_id", right_on="name", how="left")
gl_actual = gl_actual.merge(df_vendor[["name", "vendor_id"]], left_on="vendor_id", right_on="name", how="left")
gl_actual = gl_actual.merge(df_customer[["name", "customer_id"]], left_on="customer_id", right_on="name", how="left")
df_account.rename(columns={"name": "account_name"}, inplace=True)
gl_actual = gl_actual.merge(df_account[["account_name", "AccountKey"]], left_on="account_id", right_on="account_name", how="left")
gl_actual["amount"] = gl_actual["amount"].astype(float)*10
gl_actual.drop(columns=["account_id", "name_x", "name_y", "name",], inplace=True)
gl_actual.rename(columns={"product_id": "product_name", "service_id": "service_name", "procurement_id": "procurement_name",
                          "vendor_id_x": "vendor_name", "customer_id_x": "customer_name",
                          "vendor_id_y": "vendor_id", "customer_id_y": "customer_id", 
                          "companyname": "company_name", "companycode": "company_code",
                          "AccountKey": "account_id", "bu_id": "business_unit", "ammount": "amount_DKK"
                          }, inplace=True)
reordered_columns = ["date", "company_name", "company_code", "business_unit", "account_id", "account_name",
                     "vendor_id", "vendor_name", "customer_id", "customer_name",
                     "product_name", "service_name", "procurement_name"]
gl_actual = gl_actual[reordered_columns + [col for col in gl_actual.columns if col not in reordered_columns]]
gl_actual.sort_values(by=["date"], inplace=True)
gl_actual

C:\Users\jsteensgaard\AppData\Local\Temp\ipykernel_33460\2957253624.py:1: DtypeWarning: Columns (0,1,6,7,8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  gl = pd.read_csv(f"data/outputdata/fact/general_ledger.csv")
C:\Users\jsteensgaard\AppData\Local\Temp\ipykernel_33460\2957253624.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gl_actual.drop(columns=["version", "debit_credit"], inplace=True)


,date,company_name,company_code,business_unit,account_id,account_name,vendor_id,vendor_name,customer_id,customer_name,product_name,service_name,procurement_name,document_number,amount,quantity
517104,2024-01-01,Lego France,1003,LEGO_FR_RETAIL,4001,Retail Gross Sales,NaN,NaN,10.0,Target,LEGO Storage Boxes,NaN,NaN,78002109,3424.6,3.0
462019,2024-01-01,Lego France,1003,LEGO_FR_RETAIL,4003,Retail COS,64.0,Staples Solutions DK,NaN,NaN,NaN,NaN,Packing tape (roll),78001190,-485.1,18.0
7397,2024-01-01,Lego France,1003,LEGO_FR_RETAIL,4003,Retail COS,67.0,Rubberstamp Company DK,NaN,NaN,NaN,NaN,Custom print labels (LEGO logo),78001133,-119.0,7.0
550755,2024-01-01,Lego France,1003,LEGO_FR_RETAIL,4001,Retail Gross Sales,NaN,NaN,10.0,Target,LEGO Technic Pull-Back Cars,NaN,NaN,78002287,28552.4,14.0
53009,2024-01-01,Lego France,1003,LEGO_FR_RETAIL,4003,Retail COS,67.0,Rubberstamp Company DK,NaN,NaN,NaN,NaN,Custom print labels (LEGO logo),78001110,-56.1,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407685,2025-12-31,Lego France,1003,LEGO_FR_RETAIL,4003,Retail COS,64.0,Staples Solutions DK,NaN,NaN,NaN,NaN,Sticky notes (pack),78001172,-96.4,4.0
237498,2025-12-31,Lego France,1003,LEGO_FR_RETAIL,4003,Retail COS,63.0,Lyreco Denmark,NaN,NaN,NaN,NaN,Dust masks (FFP2),78001006,-34.6,3.0
191190,2025-12-31,Lego France,1003,LEGO_FR_RETAIL,4003,Retail COS,63.0,Lyreco Denmark,NaN,NaN,NaN,NaN,Disposable nitrile gloves (industrial),78001030,-53.9,6.0
66721,2025-12-31,Lego France,1003,LEGO_FR_RETAIL,4003,Retail COS,67.0,Rubberstamp Company DK,NaN,NaN,NaN,NaN,Custom print labels (LEGO logo),78001096,-55.2,5.0


In [83]:
UNIQUE_COMPANY_NAMES = gl_actual["company_name"].unique().tolist()
print(UNIQUE_COMPANY_NAMES)

['Lego France', 'Lego Germany', 'Lego Denmark', 'Lego Sweden']


In [3]:
now = dt.now()
version_tag = (
    company_name.lower() 
    + now.strftime("%d") + "/" + now.strftime("%m") + ". Kl: " + now.strftime("%H") + ":" + now.strftime("%M")
)
execute_db_operations(version_tag)

[INFO] Inserting 20 rows into dim_department using to_sql...
[INFO] Inserting 59 rows into dim_customer using to_sql...
[INFO] Inserting 150 rows into dim_product using to_sql...
[INFO] Inserting 109 rows into dim_account using to_sql...
[INFO] Inserting 120 rows into dim_procurement using to_sql...
[INFO] Inserting 100 rows into dim_service using to_sql...
[INFO] Inserting 14 rows into dim_line using to_sql...
[INFO] Inserting 69 rows into dim_vendor using to_sql...
[INFO] Inserting 9 rows into dim_bu using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)


[INFO] Inserting 400 rows into dim_employee using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)


[INFO] Inserting 134400 rows into fact_payroll using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:189: DtypeWarning: Columns (0,1,6,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_erp = pd.read_csv("data/outputdata/fact/general_ledger.csv")
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_opti

[INFO] Inserting 232327 rows into fact_general_ledger using to_sql...
[INFO] Inserting 1050 rows into fact_reporting using to_sql...


In [110]:
gl_actual_new_actual

,document_number,debit_credit,date,amount,quantity,business_unit,vendor_id,customer_id,account_id,procurement_id,...,vendor_name,vendor_segment,customer_name,customer_segment,service_name,procurement_name,product_name,unit_price,companyname,companycode
0,78001000,Credit,2025-06-22,-458269.3,7.0,LEGO_DE_ASSEMBLY,20.0,NaN,4003,1000.0,...,BASF Plastics,Raw Materials,NaN,NaN,NaN,ABS resin pellets,NaN,NaN,Lego Germany,1002
1,78001160,Credit,2024-11-20,-458611.3,8.0,LEGO_DE_ASSEMBLY,20.0,NaN,4003,1000.0,...,BASF Plastics,Raw Materials,NaN,NaN,NaN,ABS resin pellets,NaN,NaN,Lego Germany,1002
2,78001078,Credit,2024-12-11,-229576.0,8.0,LEGO_DE_ASSEMBLY,20.0,NaN,4003,1000.0,...,BASF Plastics,Raw Materials,NaN,NaN,NaN,ABS resin pellets,NaN,NaN,Lego Germany,1002
3,78001090,Credit,2025-07-31,-229294.1,4.0,LEGO_DE_ASSEMBLY,20.0,NaN,4003,1000.0,...,BASF Plastics,Raw Materials,NaN,NaN,NaN,ABS resin pellets,NaN,NaN,Lego Germany,1002
4,78001066,Credit,2024-07-11,-229284.5,6.0,LEGO_DE_ASSEMBLY,20.0,NaN,4003,1000.0,...,BASF Plastics,Raw Materials,NaN,NaN,NaN,ABS resin pellets,NaN,NaN,Lego Germany,1002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801468,78001099,Credit,2024-04-29,-270972.1,9.0,LEGO_DE_ENGINEERING,NaN,65.0,4007,NaN,...,NaN,NaN,LEGO_DE_ASSEMBLY,Intercompany,Annual Workplace Ergonomics Assessment,NaN,NaN,NaN,Lego Germany,1002
801469,78001021,Debit,2024-04-29,271119.2,4.0,LEGO_DE_ENGINEERING,NaN,65.0,4007,NaN,...,NaN,NaN,LEGO_DE_ASSEMBLY,Intercompany,Annual Workplace Ergonomics Assessment,NaN,NaN,NaN,Lego Germany,1002
801470,78001021,Credit,2024-04-29,-271119.2,4.0,LEGO_DE_ENGINEERING,NaN,65.0,4007,NaN,...,NaN,NaN,LEGO_DE_ASSEMBLY,Intercompany,Annual Workplace Ergonomics Assessment,NaN,NaN,NaN,Lego Germany,1002
801471,78001191,Debit,2024-03-19,542657.3,7.0,LEGO_DE_ENGINEERING,NaN,65.0,4007,NaN,...,NaN,NaN,LEGO_DE_ASSEMBLY,Intercompany,Annual Workplace Ergonomics Assessment,NaN,NaN,NaN,Lego Germany,1002
